## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin6_3 = pd.read_csv("data/"+'gcc6'+"_3_.csv", index_col=0)
core6_3 = pd.read_csv("data/"+"core_gcc6_3_.csv", index_col=0)

# 형태 출력
print(bin6_3.shape, core6_3.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin6_3.reset_index(inplace=True, drop=True)
core6_3.reset_index(inplace=True, drop=True)

print('reset_index 완료')
print('input data shape')
bin6_3.head()
core6_3.head()

(10820738, 2) (5120388, 2)
reset_index 완료
input data shape


,bin,label
0,86.0,1
1,83.0,0
2,232.0,0
3,185.0,0
4,18.0,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin6_3['bin'].unique()))
print(len(core6_3['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin6_3['label'].value_counts())
print(core6_3['label'].value_counts())

256
256
0    10794970
1       25768
Name: label, dtype: int64
0    5109140
1      11248
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin6_3[bin6_3['label']==1].index  # 407, 474 ...
idx_core = core6_3[core6_3['label']==1].index  # 407, 474 ...

ls_bin = list(idx_bin)
ls_core = list(idx_core)

# 최종 뽑을 행에 대한 index
ls_idx_bin, ls_idx_core = [], [] 

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능
    ls_idx_core.extend(list(idx_core + k))

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬
ls_idx_core.sort() # 인덱스 정렬


# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin6_3), ls_idx_bin))
ls_idx_core = list(filter(lambda x: x<len(core6_3), ls_idx_core))
print(len(ls_idx_bin), len(ls_idx_core))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
sub_core = len(ls_idx_core)%(right_idx)
print('나머지', sub_bin, sub_core)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
ls_idx_core = ls_idx_core[:len(ls_idx_core)-sub_core]
print('최종 길이', len(ls_idx_bin), len(ls_idx_core))

print('bin6_3', len(ls_idx_bin), len(ls_idx_core))

# loc 로 수정필요
bin6_3_Ngram = bin6_3.loc[ls_idx_bin,:].copy()
core6_3_Ngram = core6_3.loc[ls_idx_core,:].copy()

824576 359936
나머지 0 0
최종 길이 824576 359936
bin6_3 824576 359936


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin6_3_Ngram)/right_idx
count_bin = 0

goal_core = len(core6_3_Ngram)/right_idx
count_core = 0

print(goal_bin, goal_core)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin6_3.columns)
d_core = pd.DataFrame(columns = core6_3.columns)

binutils_df, coreutils_df = [], []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin) and (count_core == goal_core):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin6_3)-right_idx)
    random_idx_core = np.random.randint(len(core6_3)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin, random_idx_core)

    df_bin = bin6_3[random_idx_bin : random_idx_bin + right_idx]
    df_core = core6_3[random_idx_core : random_idx_core + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1
        
    if (1 not in df_core['label']) and count_core <goal_core:
        coreutils_df.append(df_core)
        count_core+=1

print('완료')
print(len(binutils_df), len(coreutils_df))

25768.0 11248.0
0 164681 2665567
1000 1722190 3590326
2000 9553513 5012495
3000 10206176 5019383
4000 555459 741933
5000 8176163 4857853
6000 778717 3429466
7000 2513351 4709543
8000 5472509 3567972
9000 3070154 1416194
10000 8608253 3146755
11000 6428591 2754682
12000 7997756 2222491
13000 7027561 647504
14000 1282849 3246893
15000 6074273 4327070
16000 8752656 2604951
17000 960584 1852442
18000 4779749 4337077
19000 10313425 406257
20000 1941661 338793
21000 7875588 712554
22000 5844884 1666194
23000 6976933 2607456
24000 2972915 4813322
25000 5718798 278195
완료
25768 11248


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df) + len(coreutils_df))
print(bin6_3['label'].value_counts()[1] + core6_3['label'].value_counts()[1])

37016
37016


In [6]:
f_bin_data = pd.concat(binutils_df)
f_core_data = pd.concat(coreutils_df)
f_data = pd.concat([f_bin_data, f_core_data], axis=0)
f_data.shape

(1184512, 2)

## (5) False Data + True Data 합치기

In [7]:
final = pd.concat([f_data, bin6_3_Ngram])
final.shape

(2009088, 2)

## (6) one hot encoding

In [8]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc6_3_onehot_Ngram = pd.get_dummies(final['bin'])
bc6_3_onehot_Ngram = pd.concat([final['label'], bc6_3_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc6_3_onehot_Ngram.shape)

원핫인코딩완료
(2009088, 257)


In [9]:
# 훈련 데이터, 훈련 라벨
x_bc6_3 = bc6_3_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc6_3 = bc6_3_onehot_Ngram['label'].to_numpy()
print(x_bc6_3.shape, x_bc6_3.shape)

x_bc6_3 = x_bc6_3.reshape(-1, right_idx, x_bc6_3.shape[1])
y_bc6_3 = y_bc6_3.reshape(-1, right_idx, 1)

print(x_bc6_3.shape, y_bc6_3.shape)

(2009088, 256) (2009088, 256)
(62784, 32, 256) (62784, 32, 1)


In [10]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc6_3.shape[0])

x_bc6_3 = x_bc6_3[p]
y_bc6_3 = y_bc6_3[p]

print(x_bc6_3.shape, y_bc6_3.shape)

(62784, 32, 256) (62784, 32, 1)


## (7) 모델

In [12]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [14]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc6_3, y_bc6_3):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc6_3[train],
               y_bc6_3[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc6_3[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc6_3[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 56505 samples
Epoch 1/10
56505/56505 [==============================] - 16s 276us/sample - loss: 0.0086 - accuracy: 0.9973
Epoch 2/10
56505/56505 [==============================] - 13s 223us/sample - loss: 0.0031 - accuracy: 0.9990
Epoch 3/10
56505/56505 [==============================] - 13s 226us/sample - loss: 0.0021 - accuracy: 0.9993
Epoch 

accuracy_score 0.9996765010351967
recall_score 0.9898958971218617
precision_score 0.9901990811638591
f1_score 0.9900474659317103
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 56505 samples
Epoch 1/10
56505/56505 [==============================] - 16s 288us/sample - loss: 5.5027e-04 - accuracy: 0.9999
Epoch 2/10
56505/56505 [==============================] - 56s 998us/sample - loss: 4.6061e-

accuracy_score 0.9998556696926262
recall_score 0.9938650306748467
precision_score 0.997229916897507
f1_score 0.9955446305115994
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 56505 samples
Epoch 1/10
56505/56505 [==============================] - 32s 565us/sample - loss: 2.4869e-04 - accuracy: 0.9999
Epoch 2/10
56505/56505 [==============================] - 24s 421us/sample - loss: 1.9651e-0

accuracy_score 0.9999303232998885
recall_score 0.9990831295843521
precision_score 0.9966463414634147
f1_score 0.9978632478632479
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 56505 samples
Epoch 1/10
56505/56505 [==============================] - 31s 555us/sample - loss: 7.3842e-05 - accuracy: 1.0000
Epoch 2/10
56505/56505 [==============================] - 24s 433us/sample - loss: 5.6471e-

accuracy_score 0.999970138557095
recall_score 0.9991082045184304
precision_score 0.9991082045184304
f1_score 0.9991082045184304
Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 56506 samples
Epoch 1/10
56506/56506 [==============================] - 32s 574us/sample - loss: 1.9423e-05 - accuracy: 1.0000
Epoch 2/10
56506/56506 [==============================] - 24s 432us/sample - loss: 7.4994e-0

...

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 56506 samples
Epoch 1/10
56506/56506 [==============================] - 30s 527us/sample - loss: 1.7235e-09 - accuracy: 1.0000
Epoch 2/10
56506/56506 [==============================] - 24s 423us/sample - loss: 1.3981e-09 - accuracy: 1.0000
Epoch 3/10
56506/56506 [==============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 56506 samples
Epoch 1/10
56506/56506 [==============================] - 33s 578us/sample - loss: 8.6411e-10 - accuracy: 1.0000
Epoch 2/10
56506/56506 [==============================] - 24s 431us/sample - loss: 8.0415e-10 - accuracy: 1.0000
Epoch 3/10
56506/56506 [==============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 56506 samples
Epoch 1/10
56506/56506 [==============================] - 31s 553us/sample - loss: 5.9440e-10 - accuracy: 1.0000
Epoch 2/10
56506/56506 [==============================] - 25s 439us/sample - loss: 5.6859e-10 - accuracy: 1.0000
Epoch 3/10
56506/56506 [==============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 56506 samples
Epoch 1/10
56506/56506 [==============================] - 30s 534us/sample - loss: 4.4827e-10 - accuracy: 1.0000
Epoch 2/10
56506/56506 [==============================] - 24s 431us/sample - loss: 4.3011e-10 - accuracy: 1.0000
Epoch 3/10
56506/56506 [=============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9996765010351967, 0.9998556696926262, 0.9999303232998885, 0.999970138557095, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Recall: [0.9898958971218617, 0.9938650306748467, 0.9990831295843521, 0.9991082045184304, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9901990811638591, 0.997229916897507, 0.9966463414634147, 0.9991082045184304, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9900474659317103, 0.9955446305115994, 0.9978632478632479, 0.9991082045184304, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


## (9) 평가지표

In [15]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999432632584806
10-Fold Cross_validation. Recall : 0.998195226189949
10-Fold Cross_validation. Precision : 0.998318354404321
10-Fold Cross_validation. F1-Score : 0.9982563548824988
